In [1]:
import warnings
warnings.filterwarnings("ignore")

from matplotlib import pyplot as plt

import numpy as np

import plotly.graph_objects as go
import plotly.express as px

import pandas as pd
pd.set_option("display.max_rows", None, "display.max_columns",
              None, 'display.max_colwidth', None)

import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"

In [2]:
path_dataset = os.path.join(os.path.dirname(os.getcwd()), 'Dataset')

path_result = os.path.join(os.path.dirname(os.getcwd()), 'Result')
if not os.path.exists(path_result):
    os.makedirs(path_result)

path_general = os.path.join(path_result, 'General')
if not os.path.exists(path_general):
    os.makedirs(path_general)

path_challenge = os.path.join(path_result, 'Challenge')
if not os.path.exists(path_challenge):
    os.makedirs(path_challenge)

path_solution = os.path.join(path_result, 'Solution')
if not os.path.exists(path_solution):
    os.makedirs(path_solution)

path_challenge_information = os.path.join(path_challenge, 'Information')
if not os.path.exists(path_challenge_information):
    os.makedirs(path_challenge_information)

path_solution_information = os.path.join(path_solution, 'Information')
if not os.path.exists(path_solution_information):
    os.makedirs(path_solution_information)

path_challenge_evolution = os.path.join(path_challenge, 'Evolution')
if not os.path.exists(path_challenge_evolution):
    os.makedirs(path_challenge_evolution)

path_solution_evolution = os.path.join(path_solution, 'Evolution')
if not os.path.exists(path_solution_evolution):
    os.makedirs(path_solution_evolution)

In [4]:
df_topics = pd.read_json(os.path.join(path_general, 'topics.json'))
df_topics = df_topics[df_topics['Solution_topic'] > -1]

# as if we assign the topic id as the label
label_challenge_original = df_topics['Challenge_topic'].unique().tolist()
label_challenge_refined = [f'c_{label}' for label in label_challenge_original]
label_challenge_map = dict(
    zip(label_challenge_original, label_challenge_refined))

label_solution_original = df_topics['Solution_topic'].unique().tolist()
label_solution_refined = [f's_{label}' for label in label_solution_original]
label_solution_map = dict(zip(label_solution_original, label_solution_refined))

df_topics = df_topics.replace(
    {'Challenge_topic': label_challenge_map, 'Solution_topic': label_solution_map})

categories = ['Challenge_topic', 'Solution_topic']
df_topics = df_topics.groupby(categories).size().reset_index(name='value')

# we only visualize large topics
df_topics = df_topics[df_topics['value'] > 20]

newDf = pd.DataFrame()
for i in range(len(categories)-1):
    tempDf = df_topics[[categories[i], categories[i+1], 'value']]
    tempDf.columns = ['source', 'target', 'value']
    newDf = pd.concat([newDf, tempDf])
newDf = newDf.groupby(['source', 'target']).agg({'value': 'sum'}).reset_index()

label = list(np.unique(df_topics[categories].values))
source = newDf['source'].apply(lambda x: label.index(x))
target = newDf['target'].apply(lambda x: label.index(x))
value = newDf['value']

link = dict(source=source, target=target, value=value)
node = dict(label=label)
data = go.Sankey(link=link, node=node)

fig = go.Figure(data)
fig.update_layout(height=1000, width=1000, font=dict(size=30))
fig.write_image(os.path.join(path_challenge_information,
                'Challenge solution sankey.png'))

In [8]:
# Create challenge topic distribution tree map

df_topics = pd.read_json(os.path.join(path_general, 'Topics.json'))
df_topics['Challenge_participation_count'] = df_topics['Challenge_answer_count'] + \
    df_topics['Challenge_comment_count']

fig = px.treemap(
    df_topics,
    path=['Tool', 'Platform'],
    values='Challenge_participation_count',
    color='Challenge_topic',
    width=2000,
    height=1000,
)
fig.write_image(os.path.join(path_challenge_information,
                'Challenge_topic_distribution.png'))


In [9]:
# Create solution topic distribution tree map

df_topics = pd.read_json(os.path.join(path_general, 'Topics.json'))
df_topics = df_topics[df_topics['Solution_topic'] > -1]
df_topics['Challenge_participation_count'] = df_topics['Challenge_answer_count'] + \
    df_topics['Challenge_comment_count']

fig = px.treemap(
    df_topics,
    path=['Tool', 'Platform'],
    values='Challenge_participation_count',
    color='Solution_topic',
    width=2000,
    height=1000,
)
fig.write_image(os.path.join(path_solution_information,
                'Solution_topic_distribution.png'))


In [10]:
# Collect challenge statistics information

df_challenge = pd.read_json(os.path.join(path_general, 'Topics.json'))
df_challenge = df_challenge[df_challenge['Challenge_topic'] > -1]

df_challenge['Challenge_comment_count'] = df_challenge['Challenge_comment_count'].fillna(0)
df_challenge['Challenge_solved_time'] = df_challenge['Challenge_closed_time'] - \
    df_challenge['Challenge_creation_time']
df_challenge['Challenge_adjusted_solved_time'] = df_challenge['Solution_last_edit_time'] - \
    df_challenge['Challenge_last_edit_time']
df_challenge['Challenge_participation_count'] = df_challenge['Challenge_answer_count'] + \
    df_challenge['Challenge_comment_count']

total_count = df_challenge['Challenge_topic'].count()
df_topics = []

for name, group in df_challenge.groupby('Challenge_topic'):
    Mean_score = group['Challenge_score'].mean()
    Mean_favorite_count = group['Challenge_favorite_count'].mean()
    Mean_follower_count = group['Challenge_follower_count'].mean()
    Mean_link_count = group['Challenge_link_count'].mean()
    Mean_information_entropy = group['Challenge_information_entropy'].mean()
    Mean_readability = group['Challenge_readability'].mean()
    Mean_sentence_count = group['Challenge_sentence_count'].mean()
    Mean_word_count = group['Challenge_word_count'].mean()
    Mean_unique_word_count = group['Challenge_unique_word_count'].mean()
    Mean_view_count = group['Challenge_view_count'].mean()
    Mean_answer_count = group['Challenge_answer_count'].mean()
    Mean_comment_count = group['Challenge_comment_count'].mean()
    Mean_participation_count = Mean_answer_count + Mean_comment_count
    Score_participation_ratio = Mean_score / Mean_participation_count
    Score_participation_weighted_product = (
        group['Challenge_score'] * group['Challenge_participation_count']).mean()
    Count = group['Challenge_topic'].count()
    Count_ratio = Count / total_count * 100
    Solved_ratio = group['Challenge_closed_time'].notna().sum() / Count
    Mean_solved_time = group['Challenge_solved_time'].mean(
    ) / pd.Timedelta(hours=1)
    Median_solved_time = group['Challenge_solved_time'].median(
    ) / pd.Timedelta(hours=1)
    Mean_adjusted_solved_time = group['Challenge_adjusted_solved_time'].mean(
    ) / pd.Timedelta(hours=1)
    Median_adjusted_solved_time = group['Challenge_adjusted_solved_time'].median(
    ) / pd.Timedelta(hours=1)
    topic_info = {
        'Challenge_topic': name,
        'Mean_score': Mean_score,
        'Mean_favorite_count': Mean_favorite_count,
        'Mean_follower_count': Mean_follower_count,
        'Mean_link_count': Mean_link_count,
        'Mean_information_entropy': Mean_information_entropy,
        'Mean_readability': Mean_readability,
        'Mean_sentence_count': Mean_sentence_count,
        'Mean_word_count': Mean_word_count,
        'Mean_unique_word_count': Mean_unique_word_count,
        'Mean_view_count': Mean_view_count,
        'Mean_answer_count': Mean_answer_count,
        'Mean_comment_count': Mean_comment_count,
        'Score_participation_ratio': Score_participation_ratio,
        'Score_participation_weighted_product': Score_participation_weighted_product,
        'Count_ratio': Count_ratio,
        'Solved_ratio': Solved_ratio,
        'Mean_solved_time': Mean_solved_time,
        'Median_solved_time': Median_solved_time,
        'Mean_adjusted_solved_time': Mean_adjusted_solved_time,
        'Median_adjusted_solved_time': Median_adjusted_solved_time,
    }
    df_topics.append(topic_info)

df_topics = pd.DataFrame(df_topics)
df_topics.to_json(os.path.join(path_challenge_information,
                  'general.json'), indent=4, orient='records')
df_topics = df_topics.set_index('Challenge_topic')

fig = df_topics.sort_values('Mean_score', ascending=False)['Mean_score'].plot(
    kind='bar', figsize=(15, 8), title='Challenge mean score', rot=15).get_figure()
fig.savefig(os.path.join(path_challenge_information, 'Mean_score.png'))
plt.close()

fig = df_topics.sort_values('Mean_favorite_count', ascending=False)['Mean_favorite_count'].plot(
    kind='bar', figsize=(15, 8), title='Challenge mean favorite count', rot=15).get_figure()
fig.savefig(os.path.join(path_challenge_information, 'Mean_favorite_count.png'))
plt.close()

fig = df_topics.sort_values('Mean_follower_count', ascending=False)['Mean_follower_count'].plot(
    kind='bar', figsize=(15, 8), title='Challenge mean follower count', rot=15).get_figure()
fig.savefig(os.path.join(path_challenge_information, 'Mean_follower_count.png'))
plt.close()

fig = df_topics.sort_values('Mean_link_count', ascending=False)['Mean_link_count'].plot(
    kind='bar', figsize=(15, 8), title='Challenge mean link count', rot=15).get_figure()
fig.savefig(os.path.join(path_challenge_information, 'Mean_link_count.png'))
plt.close()

fig = df_topics.sort_values('Mean_information_entropy', ascending=False)['Mean_information_entropy'].plot(
    kind='bar', figsize=(15, 8), title='Challenge mean info entropy', rot=15).get_figure()
fig.savefig(os.path.join(path_challenge_information, 'Mean_information_entropy.png'))
plt.close()

fig = df_topics.sort_values('Mean_readability', ascending=False)['Mean_readability'].plot(
    kind='bar', figsize=(15, 8), title='Challenge mean readability', rot=15).get_figure()
fig.savefig(os.path.join(path_challenge_information, 'Mean_readability.png'))
plt.close()

fig = df_topics.sort_values('Mean_sentence_count', ascending=False)['Mean_sentence_count'].plot(
    kind='bar', figsize=(15, 8), title='Challenge mean sentence count', rot=15).get_figure()
fig.savefig(os.path.join(path_challenge_information, 'Mean_sentence_count.png'))
plt.close()

fig = df_topics.sort_values('Mean_word_count', ascending=False)['Mean_word_count'].plot(
    kind='bar', figsize=(15, 8), title='Challenge mean word count', rot=15).get_figure()
fig.savefig(os.path.join(path_challenge_information, 'Mean_word_count.png'))
plt.close()

fig = df_topics.sort_values('Mean_unique_word_count', ascending=False)['Mean_unique_word_count'].plot(
    kind='bar', figsize=(15, 8), title='Challenge mean unique word count', rot=15).get_figure()
fig.savefig(os.path.join(path_challenge_information, 'Mean_unique_word_count.png'))
plt.close()

fig = df_topics.sort_values('Mean_view_count', ascending=False)['Mean_view_count'].plot(
    kind='bar', figsize=(15, 8), title='Challenge mean view count', rot=15).get_figure()
fig.savefig(os.path.join(path_challenge_information, 'Mean_view_count.png'))
plt.close()

fig = df_topics.sort_values('Mean_answer_count', ascending=False)['Mean_answer_count'].plot(
    kind='bar', figsize=(15, 8), title='Challenge mean answer count', rot=15).get_figure()
fig.savefig(os.path.join(path_challenge_information, 'Mean_answer_count.png'))
plt.close()

fig = df_topics.sort_values('Mean_comment_count', ascending=False)['Mean_comment_count'].plot(
    kind='bar', figsize=(15, 8), title='Challenge mean comment count', rot=15).get_figure()
fig.savefig(os.path.join(path_challenge_information, 'Mean_comment_count.png'))
plt.close()

fig = df_topics.sort_values('Score_participation_ratio', ascending=False)['Score_participation_ratio'].plot(
    kind='bar', figsize=(15, 8), title='Challenge score participation ratio', rot=15).get_figure()
fig.savefig(os.path.join(path_challenge_information,
            'Score_participation_ratio.png'))
plt.close()

fig = df_topics.sort_values('Score_participation_weighted_product', ascending=False)['Score_participation_weighted_product'].plot(
    kind='bar', figsize=(15, 8), title='Challenge score participation weighted product', rot=15).get_figure()
fig.savefig(os.path.join(path_challenge_information,
            'Score_participation_weighted_product.png'))
plt.close()

fig = df_topics.sort_values('Count_ratio', ascending=False)['Count_ratio'].plot(
    kind='bar', figsize=(15, 8), title='Challenge count ratio', rot=15).get_figure()
fig.savefig(os.path.join(path_challenge_information, 'Count_ratio.png'))
plt.close()

fig = df_topics.sort_values('Solved_ratio')['Solved_ratio'].plot(
    kind='bar', figsize=(15, 8), title='Challenge Solved ratio', rot=15).get_figure()
fig.savefig(os.path.join(path_challenge_information, 'solved_ratio.png'))
plt.close()

fig = df_topics.sort_values('Mean_solved_time', ascending=False)['Mean_solved_time'].plot(
    kind='bar', figsize=(15, 8), title='Challenge median solved time', rot=15).get_figure()
fig.savefig(os.path.join(path_challenge_information, 'mean_solved_time.png'))
plt.close()

fig = df_topics.sort_values('Median_solved_time', ascending=False)['Median_solved_time'].plot(
    kind='bar', figsize=(15, 8), title='Challenge mean solved time', rot=15).get_figure()
fig.savefig(os.path.join(path_challenge_information, 'median_solved_time.png'))
plt.close()

fig = df_topics.sort_values('Mean_adjusted_solved_time', ascending=False)['Mean_adjusted_solved_time'].plot(
    kind='bar', figsize=(15, 8), title='Challenge mean adjusted solved time', rot=15).get_figure()
fig.savefig(os.path.join(path_challenge_information, 'Mean_adjusted_solved_time.png'))
plt.close()

fig = df_topics.sort_values('Median_adjusted_solved_time', ascending=False)['Median_adjusted_solved_time'].plot(
    kind='bar', figsize=(15, 8), title='Challenge median adjusted solved time', rot=15).get_figure()
fig.savefig(os.path.join(path_challenge_information, 'Median_adjusted_solved_time.png'))
plt.close()


In [11]:
# Collect solution statistics information

df_solution = pd.read_json(os.path.join(path_general, 'Topics.json'))
df_solution = df_solution[df_solution['Solution_topic'] > -1]

total_count = df_challenge['Solution_topic'].count()
df_topics = []

for name, group in df_challenge.groupby('Solution_topic'):
    Mean_score = group['Solution_score'].mean()
    Mean_link_count = group['Solution_link_count'].mean()
    Mean_information_entropy = group['Solution_information_entropy'].mean()
    Mean_readability = group['Solution_readability'].mean()
    Mean_sentence_count = group['Solution_sentence_count'].mean()
    Mean_word_count = group['Solution_word_count'].mean()
    Mean_unique_word_count = group['Solution_unique_word_count'].mean()
    Mean_comment_count = group['Solution_comment_count'].mean()
    Count_ratio = group['Solution_topic'].count() / total_count * 100
    topic_info = {
        'Solution_topic': name,
        'Mean_score': Mean_score,
        'Mean_link_count': Mean_link_count,
        'Mean_information_entropy': Mean_information_entropy,
        'Mean_readability': Mean_readability,
        'Mean_sentence_count': Mean_sentence_count,
        'Mean_word_count': Mean_word_count,
        'Mean_unique_word_count': Mean_unique_word_count,
        'Mean_comment_count': Mean_comment_count,
        'Count_ratio': Count_ratio,
    }
    df_topics.append(topic_info)

df_topics = pd.DataFrame(df_topics)
df_topics.to_json(os.path.join(path_solution_information,
                  'general.json'), indent=4, orient='records')
df_topics = df_topics.set_index('Solution_topic')

fig = df_topics.sort_values('Mean_score', ascending=False)['Mean_score'].plot(
    kind='bar', figsize=(15, 8), title='Solution mean score', rot=15).get_figure()
fig.savefig(os.path.join(path_solution_information, 'Mean_score.png'))
plt.close()

fig = df_topics.sort_values('Mean_link_count', ascending=False)['Mean_link_count'].plot(
    kind='bar', figsize=(15, 8), title='Solution mean link count', rot=15).get_figure()
fig.savefig(os.path.join(path_solution_information, 'Mean_link_count.png'))
plt.close()

fig = df_topics.sort_values('Mean_information_entropy', ascending=False)['Mean_information_entropy'].plot(
    kind='bar', figsize=(15, 8), title='Solution mean info entropy', rot=15).get_figure()
fig.savefig(os.path.join(path_solution_information, 'Mean_information_entropy.png'))
plt.close()

fig = df_topics.sort_values('Mean_readability', ascending=False)['Mean_readability'].plot(
    kind='bar', figsize=(15, 8), title='Solution mean readability', rot=15).get_figure()
fig.savefig(os.path.join(path_solution_information, 'Mean_readability.png'))
plt.close()

fig = df_topics.sort_values('Mean_sentence_count', ascending=False)['Mean_sentence_count'].plot(
    kind='bar', figsize=(15, 8), title='Solution mean sentence count', rot=15).get_figure()
fig.savefig(os.path.join(path_solution_information, 'Mean_sentence_count.png'))
plt.close()

fig = df_topics.sort_values('Mean_word_count', ascending=False)['Mean_word_count'].plot(
    kind='bar', figsize=(15, 8), title='Solution mean word count', rot=15).get_figure()
fig.savefig(os.path.join(path_solution_information, 'Mean_word_count.png'))
plt.close()

fig = df_topics.sort_values('Mean_unique_word_count', ascending=False)['Mean_unique_word_count'].plot(
    kind='bar', figsize=(15, 8), title='Solution mean unique word count', rot=15).get_figure()
fig.savefig(os.path.join(path_solution_information, 'Mean_unique_word_count.png'))
plt.close()

fig = df_topics.sort_values('Mean_comment_count', ascending=False)['Mean_comment_count'].plot(
    kind='bar', figsize=(15, 8), title='Solution mean comment count', rot=15).get_figure()
fig.savefig(os.path.join(path_solution_information, 'Mean_comment_count.png'))
plt.close()

fig = df_topics.sort_values('Count_ratio', ascending=False)['Count_ratio'].plot(
    kind='bar', figsize=(15, 8), title='Solution count ratio', rot=15).get_figure()
fig.savefig(os.path.join(path_solution_information, 'Count_ratio.png'))
plt.close()


In [12]:
import scipy.interpolate
from statsmodels.nonparametric.smoothers_lowess import lowess as sm_lowess


def smooth(x, y, xgrid, lowess_kw=None):
    samples = np.random.choice(len(x), 50, replace=True)
    y_s = y[samples]
    x_s = x[samples]
    y_sm = sm_lowess(y_s, x_s, **lowess_kw)
    # regularly sample it onto the grid
    y_grid = scipy.interpolate.interp1d(
        x_s, y_sm, fill_value='extrapolate')(xgrid)
    return y_grid


def lowess_with_confidence_bounds(x, y, conf_interval=0.95, lowess_kw=None):
    """
    Perform Lowess regression and determine a confidence interval by bootstrap resampling
    """
    xgrid = np.linspace(x.min(), x.max())

    K = 100
    smooths = np.stack([smooth(x, y, xgrid, lowess_kw) for _ in range(K)]).T

    mean = np.nanmean(smooths, axis=1)
    stderr = scipy.stats.sem(smooths, axis=1)

    clower = np.nanpercentile(smooths, (1-conf_interval)*50, axis=1)
    cupper = np.nanpercentile(smooths, (1+conf_interval)*50, axis=1)

    return xgrid, mean, stderr, clower, cupper

In [13]:
df_all = pd.read_json(os.path.join(path_general, 'Topics.json'))
df_challenge = df_all[df_all['Challenge_topic'] > -1]
# BigQuery Stack Overflow public dataset is updated until Nov 24, 2022, 1:39:22 PM UTC-5
min(df_challenge['Challenge_creation_time']), max(df_challenge['Challenge_creation_time'])


(Timestamp('2014-08-08 14:04:22.160000'),
 Timestamp('2023-02-22 01:36:03.995000'))

In [14]:
# Explore challenge topics evolution

df_challenge = pd.read_json(os.path.join(path_general, 'Topics.json'))
df_challenge = df_challenge[df_challenge['Challenge_topic'] > -1]
df_challenge = df_challenge[(df_challenge['Challenge_creation_time'] > '2014-09-14')
                            & (df_challenge['Challenge_creation_time'] < '2022-11-21')]

for name, group in df_challenge.groupby('Challenge_topic'):
    group = group.groupby(pd.Grouper(key='Challenge_creation_time', freq='2W')).agg(
        Count=('Challenge_topic', 'count')).reset_index()
    x = pd.to_datetime(group['Challenge_creation_time']).values
    x = np.array([i.astype('datetime64[D]').astype(int) for i in x])
    y = group['Count'].values
    # 95% confidence interval
    xgrid, mean, stderr, clower, cupper = lowess_with_confidence_bounds(
        x, y, conf_interval=0.95, lowess_kw={"frac": 0.5, "it": 5, "return_sorted": False})
    x = pd.to_datetime(group['Challenge_creation_time']).values
    fig, ax = plt.subplots(figsize=(20, 10))
    plt.plot(x, y, 'k.', label='Observations')
    plt.plot(xgrid, mean, color='tomato', label='LOWESS')
    plt.fill_between(xgrid, clower, cupper, alpha=0.3,
                     label='LOWESS uncertainty')
    plt.legend(loc='best')
    fig.savefig(os.path.join(path_challenge_evolution,
                f'Topic_{name}'), bbox_inches="tight")
    plt.close()

In [15]:
df_all = pd.read_json(os.path.join(path_general, 'Topics.json'))
df_solution = df_all[df_all['Solution_topic'] > -1]
# BigQuery Stack Overflow public dataset is updated until Nov 24, 2022, 1:39:22 PM UTC-5
min(df_solution['Challenge_creation_time']), max(
    df_solution['Challenge_creation_time'])

(Timestamp('2014-09-14 22:12:24.493000'),
 Timestamp('2023-02-21 18:36:06.284000'))

In [16]:
# Explore solution topics evolution

df_solution = pd.read_json(os.path.join(path_general, 'Topics.json'))
df_solution = df_solution[df_solution['Solution_topic'] > -1]
df_solution = df_solution[(df_solution['Challenge_creation_time'] > '2014-09-14')
                          & (df_solution['Challenge_creation_time'] < '2022-11-21')]

for name, group in df_solution.groupby('Solution_topic'):
    group = group.groupby(pd.Grouper(key='Challenge_closed_time', freq='W')).agg(
        Count=('Solution_topic', 'count')).reset_index()
    x = pd.to_datetime(group['Challenge_closed_time']).values
    x = np.array([i.astype('datetime64[D]').astype(int) for i in x])
    y = group['Count'].values
    # 95% confidence interval
    xgrid, mean, stderr, clower, cupper = lowess_with_confidence_bounds(
        x, y, conf_interval=0.95, lowess_kw={"frac": 0.5, "it": 5, "return_sorted": False})
    x = pd.to_datetime(group['Challenge_closed_time']).values
    fig, ax = plt.subplots(figsize=(20, 10))
    plt.plot(x, y, 'k.', label='Observations')
    plt.plot(xgrid, mean, color='tomato', label='LOWESS')
    plt.fill_between(xgrid, clower, cupper, alpha=0.3,
                     label='LOWESS uncertainty')
    plt.legend(loc='best')
    fig.savefig(os.path.join(path_solution_evolution,
                f'Topic_{name}'), bbox_inches="tight")
    plt.close()